# Impyte Documentation
This is a first practicle attempt to clarify the usage of `impyte`. It's a collection of easy applicable and reproducible examples that you could use to simplyify your data processing workflow.

## Importing and generating sample data
In order to show some of the features of the library, we'll be using Kaggle's HR data that can be found [here](https://www.kaggle.com/ludobenistant/hr-analytics).

In [1]:
# import library and data set
import impyte
import pandas as pd
from tools.data_prep import remove_random

In [2]:
data = pd.read_csv('data/hr_test.csv')

In [3]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


## Add random missing values
In order to show some of the pattern visualization and imputation methods, we need incomplete data sets. To achieve this, we'll be using a helper function that deletes values at random. 25 % of all values from each category will be deleted.

In [4]:
data = remove_random(data, .25, randomize_index=False)

In [5]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.98,4.0,185.0,2.0,0.0,0.0,0.0,marketing,medium
1,NaN,0.68,5.0,151.0,2.0,0.0,0.0,0.0,RandD,medium
2,NaN,0.70,3.0,209.0,3.0,0.0,0.0,0.0,technical,high
3,NaN,0.89,3.0,221.0,3.0,0.0,0.0,0.0,sales,low
4,NaN,0.77,6.0,215.0,4.0,0.0,0.0,0.0,sales,medium


In [6]:
data.describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000
mean,0.613578,0.715996,3.803829,201.123624,3.499761,0.145162,0.238632,0.021197
std,0.248008,0.171067,1.232930,49.945633,1.461305,0.352277,0.426262,0.144044
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [7]:
len(data)

14999

In [8]:
reload(impyte)

<module 'impyte' from 'impyte.pyc'>

## Testing rudimentary features
Below is a first trial run of some of the features implemented by impyter and its helper classes.

## `NanChecker`
Functionality testing of `NanChecker` class.

In [9]:
nan_checker = impyte.NanChecker()

#### `NanChecker.is_nan(data, nan_vals=None, recursive=True)`
Detect missing values (NaN in numeric arrays, empty strings in string arrays). NaN detection feature also enables NaN-value ingestion as parameter values.

In [10]:
# [True, False, False]
print nan_checker.is_nan(["", 'None', 'NaN'])

# [True, True, True]
print nan_checker.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN'])

[True, False, False]
[True, True, True]


In [11]:
# Recursive nan detection
# [True, True, False, [False, True, True]]
print nan_checker.is_nan(["", None, 'NaN', ["List Value 1", '', None]])

[True, True, False, [False, True, True]]


In [12]:
# Values can be declared as nan-values
# [True, False, False, True]
nan_checker.is_nan(['NaN', 'Empty', 'None', 'N/A'], nan_vals=['NaN', 'N/A'])

[True, False, False, True]

## `Pattern`
Functionality testing of `Pattern` class. The `Pattern` class stores different patterns and data summaries regarding NaN values. 

#### `Pattern._get_discrete_and_continuous(tmpdata)`
Returns the column names of discrete and continuous variables. Column names are stored in lists for easy selection. Those lists are stored in one dictionary object. All continuous column names can be accessed through `['continuous']` all discrete variables through `['discrete']`.

In [13]:
reload(impyte)
pattern_log = impyte.Pattern()

In [14]:
pattern_log._get_discrete_and_continuous(data)

{'continuous': ['satisfaction_level',
  'last_evaluation',
  'number_project',
  'average_montly_hours',
  'time_spend_company',
  'Work_accident',
  'left',
  'promotion_last_5years'],
 'discrete': ['sales', 'salary']}

#### `Pattern._compute_pattern(data, nan_values="", verbose=False)`
Function that checks for missing values and prints out a quick table of a summary of missing values. Includes pattern overview and counts of missing values by column. Computes pattern and returns its pattern structure plus the count of data points for each of these patterns. To get a nice digestible table add `["table"]` selector to output.

In [15]:
pattern_dict = pattern_log._compute_pattern(data)
indices, table = pattern_dict["indices"], pattern_dict["table"]

In [16]:
table

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


In [17]:
indices[0][:10] # first 10 indices of pattern 0

[3740, 3741, 3742, 3743, 3744, 3745, 3746, 3747, 3748, 3749]

In [18]:
pattern_nr = 0
print "Pattern {} has {:,} rows.".format(pattern_nr, len(indices[0])) 

Pattern 0 has 11,259 rows.


#### `Pattern.get_pattern()`
Returns NaN-patterns based on primary computation or initiates new computation of NaN-patterns. Uses `_compute_patterns()` to visualize patterns if not yet computed. Otherwise returns stored information regarding patterns.

In [19]:
pattern_log.get_pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


#### `Pattern.row_nan_pattern(row)`
This is a core piece to determining which patterns are available. Other methods work with the output of `row_nan_pattern()` to determine unique pattern structures and count these before turning them into readable tables.

In [20]:
# (1, 1, 'NaN', 'NaN')
print pattern_log.row_nan_pattern(['Value 1', 'Value 2', '', None])

(1, 1, 'NaN', 'NaN')


#### `Pattern.get_pattern_indices(pattern_no)`

In [21]:
pattern_log.get_pattern_indices(0)[:10] # get first 10 indices of pattern 0

[3740, 3741, 3742, 3743, 3744, 3745, 3746, 3747, 3748, 3749]

#### `Pattern.get_continuous()`
Returns list with names of all continuous variables.

In [22]:
pattern_log.get_continuous()

['satisfaction_level',
 'last_evaluation',
 'number_project',
 'average_montly_hours',
 'time_spend_company',
 'Work_accident',
 'left',
 'promotion_last_5years']

#### `Pattern.get_discrete()`
Returns list with names of all discrete variables.

In [23]:
pattern_log.get_discrete()

['sales', 'salary']

## `Impyter`

In [49]:
reload(impyte)
imputer = impyte.Impyter() # instantiate impyte class

### Load data into imputer

In [50]:
_ = imputer.load_data(data)

In [51]:
%timeit imputer.pattern()

The slowest run took 152536.24 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 5.96 µs per loop


In [52]:
%timeit imputer.get_pattern(4)

1000 loops, best of 3: 677 µs per loop


In [106]:
reload(impyte)
imputer = impyte.Impyter(data) # instantiate impyte class
nan_checker = impyte.NanChecker()
pattern_log = impyte.Pattern()
imputer.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


In [107]:
_ = imputer.drop_pattern(4, inplace=True)

In [108]:
imputer.pattern_log.missing_per_column

[374, 374, 374, 374, 374, 374, 374, 374, 374, 374]

In [109]:
imputer.get_missing_summary()

,Missing,Complete,Percentage
satisfaction_level,374,14251,2.56 %
last_evaluation,374,14251,2.56 %
number_project,374,14251,2.56 %
average_montly_hours,374,14251,2.56 %
time_spend_company,374,14251,2.56 %
Work_accident,374,14251,2.56 %
left,374,14251,2.56 %
promotion_last_5years,374,14251,2.56 %
sales,374,14251,2.56 %
salary,374,14251,2.56 %


In [30]:
imputer.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374
10,1,1,1,1,1,1,NaN,1,1,1,374


In [31]:
_ = imputer.load_data(data)

In [32]:
imputer.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


In [33]:
_ = imputer.drop_pattern(4)

In [34]:
imputer.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


In [35]:
imputer.pattern_log.easy_access

{(1, 1, 1, 1, 1, 1, 1, 1, 1, 'NaN'): ['salary'],
 (1, 1, 1, 1, 1, 1, 1, 1, 'NaN', 1): ['sales'],
 (1, 1, 1, 1, 1, 1, 1, 'NaN', 1, 1): ['promotion_last_5years'],
 (1, 1, 1, 1, 1, 1, 'NaN', 1, 1, 1): ['left'],
 (1, 1, 1, 1, 1, 'NaN', 1, 1, 1, 1): ['Work_accident'],
 (1, 1, 1, 1, 'NaN', 1, 1, 1, 1, 1): ['time_spend_company'],
 (1, 1, 1, 'NaN', 1, 1, 1, 1, 1, 1): ['average_montly_hours'],
 (1, 1, 'NaN', 1, 1, 1, 1, 1, 1, 1): ['number_project'],
 (1, 'NaN', 1, 1, 1, 1, 1, 1, 1, 1): ['last_evaluation'],
 ('NaN', 1, 1, 1, 1, 1, 1, 1, 1, 1): ['satisfaction_level']}

### Comparison of complete indices methods

In [36]:
%timeit len(imputer.pattern_log.get_complete_indices())

1000 loops, best of 3: 869 µs per loop


In [37]:
%timeit len(imputer.get_complete_old())

100 loops, best of 3: 5.54 ms per loop


In [38]:
_ = imputer.drop_pattern(7, inplace=True)

In [39]:
data.iloc[:, imputer.pattern_log.store_tuple_columns[(1, 1, 1, 1, 1, 1, 1, 1, 1, 'NaN')]].columns

Index([u'salary'], dtype='object')

In [40]:
data.tail()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
14994,0.37,0.49,2.0,138.0,3.0,0.0,1.0,0.0,technical,low
14995,0.75,0.63,4.0,261.0,3.0,1.0,0.0,0.0,marketing,low
14996,0.39,0.68,2.0,282.0,5.0,0.0,1.0,0.0,marketing,low
14997,0.83,0.86,4.0,98.0,4.0,0.0,0.0,0.0,sales,low
14998,0.51,0.41,2.0,164.0,4.0,0.0,0.0,0.0,sales,medium


### Timing some of the functions
Below is a piece of information on efficiency of the functions. This doesn't reflect complexity of the functions, but it will show an runtime estimate for your local machine.

In [41]:
%timeit nan_checker.is_nan(["", 'None', 'NaN'])
%timeit nan_checker.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN'])

The slowest run took 4.39 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 4.78 µs per loop
100000 loops, best of 3: 4.71 µs per loop


### `one_hot_encode()`
Transforms categorical values in a one-hot-encoded data set.

In [113]:
reload(impyte)
imputer = impyte.Impyter()
imputer.load_data(data)
_ = imputer.pattern()

In [114]:
imputer.one_hot_encode()

In [121]:
cols = [x for x in imputer.data.columns if '_ohe_' in x] # select only columns with _ohe_ mark
imputer.data[cols].head()

,sales_ohe_IT,sales_ohe_RandD,sales_ohe_accounting,sales_ohe_hr,sales_ohe_management,sales_ohe_marketing,sales_ohe_product_mng,sales_ohe_sales,sales_ohe_support,sales_ohe_technical,salary_ohe_high,salary_ohe_low,salary_ohe_medium
0,0,0,0,0,0,1,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,1,1,0,0
3,0,0,0,0,0,0,0,1,0,0,0,1,0
4,0,0,0,0,0,0,0,1,0,0,0,0,1


### `one_hot_decode()`
Transforms one-hot-encoded columns back to categorical values.

In [122]:
imputer.one_hot_decode()

In [123]:
imputer.data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.98,4.0,185.0,2.0,0.0,0.0,0.0,marketing,medium
1,NaN,0.68,5.0,151.0,2.0,0.0,0.0,0.0,RandD,medium
2,NaN,0.70,3.0,209.0,3.0,0.0,0.0,0.0,technical,high
3,NaN,0.89,3.0,221.0,3.0,0.0,0.0,0.0,sales,low
4,NaN,0.77,6.0,215.0,4.0,0.0,0.0,0.0,sales,medium


### `impute()`
Impute is the core method of impyte. The method works out of the box and uses Random Forest estimators per default to impute missing values. It automatically performs cross-validation to showcase the potential accuracy of the imputation.

In [142]:
reload(impyte)
imputer = impyte.Impyter()
imputer.load_data(data)
_ = imputer.pattern()
#imputer.one_hot_encode() #error: ValueError: labels ['sales'] not contained in axis
imputer.impute()

      satisfaction_level
3740                0.75
3741                0.99
3742                0.10
3743                0.96
3744                0.24


(['satisfaction_level',
  'last_evaluation',
  'number_project',
  'average_montly_hours',
  'time_spend_company',
  'Work_accident',
  'left',
  'promotion_last_5years'],
 ['sales', 'salary'])

In [43]:
data['last_evaluation'].describe()

count    14625.000000
mean         0.715954
std          0.171263
min          0.360000
25%          0.560000
50%          0.720000
75%          0.870000
max          1.000000
Name: last_evaluation, dtype: float64

### Scaling for preprocessing

In [44]:
from sklearn.preprocessing import StandardScaler

In [45]:
scaler = StandardScaler()

In [46]:
test = data.dropna()

In [47]:
len(test[test.apply(nan_checker.is_nan)])

11259

In [48]:
test.describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,11259.000000,11259.000000,11259.000000,11259.000000,11259.000000,11259.000000,11259.000000,11259.000000
mean,0.611581,0.717509,3.807532,201.078337,3.503419,0.142997,0.240608,0.021050
std,0.248967,0.171942,1.232370,50.063233,1.456804,0.350085,0.427471,0.143557
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [75]:
test_scaled = scaler.fit_transform(test[test.corr().columns])

In [76]:
test[:5]

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
3740,0.90,0.74,5.0,249.0,3.0,0.0,0.0,0.0,support,medium
3741,0.89,0.52,4.0,189.0,3.0,0.0,0.0,0.0,support,medium
3742,0.58,0.87,3.0,268.0,2.0,0.0,0.0,0.0,sales,low
3743,0.62,0.50,4.0,156.0,2.0,0.0,0.0,0.0,support,medium
3744,0.27,0.75,5.0,264.0,3.0,0.0,0.0,0.0,technical,medium


In [83]:
pd.DataFrame(test_scaled[:5])

,0,1,2,3,4,5,6,7
0,1.158500,0.142626,0.978675,0.962305,-0.349958,-0.404773,-0.557824,-0.151309
1,1.118144,-1.143529,0.164571,-0.240267,-0.349958,-0.404773,-0.557824,-0.151309
2,-0.132901,0.902626,-0.649533,1.343119,-1.027781,-0.404773,-0.557824,-0.151309
3,0.028524,-1.260452,0.164571,-0.901681,-1.027781,-0.404773,-0.557824,-0.151309
4,-1.383945,0.201087,0.978675,1.262947,-0.349958,-0.404773,-0.557824,-0.151309


In [79]:
test_rescaled = scaler.inverse_transform(test_scaled)

In [82]:
pd.DataFrame(test_rescaled[:5])

,0,1,2,3,4,5,6,7
0,0.90,0.74,5.0,249.0,3.0,0.0,0.0,0.0
1,0.89,0.52,4.0,189.0,3.0,0.0,0.0,0.0
2,0.58,0.87,3.0,268.0,2.0,0.0,0.0,0.0
3,0.62,0.50,4.0,156.0,2.0,0.0,0.0,0.0
4,0.27,0.75,5.0,264.0,3.0,0.0,0.0,0.0


In [131]:
reload(impyte)
testdf = pd.read_csv('data/masterdf_201710230_andirs.csv', low_memory=False)
imp = impyte.Impyter()
_ = imp.load_data(testdf)
imp.pattern()

,Unnamed: 0,Incident Date,EAS,Incident_Year,Incident_Cat,Incident_Dummy,Neighborhood,Location_y,Address,Building_Cat,...,count all complaints not corrected,count potential fire control not corrected,count fire emergency safety,count potential fire cause,count fire emergency safety not corrected,count potential fire cause not corrected,next_fire_dpt_address,next_fire_dpt_distance,next_fire_dpt_latlong,Count
0,1,NaN,1,NaN,NaN,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,170517
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,24767
2,1,1,1,1,NaN,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,24


In [132]:
imp.get_missing_summary()

,Complete,Missing,Percentage
Unnamed: 0,195308,0,0.00 %
next_fire_dpt_address,195308,0,0.00 %
count potential fire cause not corrected,195308,0,0.00 %
count fire emergency safety not corrected,195308,0,0.00 %
count potential fire cause,195308,0,0.00 %
count fire emergency safety,195308,0,0.00 %
count potential fire control not corrected,195308,0,0.00 %
count all complaints not corrected,195308,0,0.00 %
count all complaints,195308,0,0.00 %
count potential fire control,195308,0,0.00 %


In [133]:
imp.get_missing_summary(importance_filter=True)

,Complete,Missing,Percentage
Incident_Year,24791,170517,87.31 %
Incident Date,24791,170517,87.31 %
Incident_Cat,24767,170541,87.32 %
